# HW4 Trees and Forests

Official instructions:

https://www.cs.tufts.edu/cs/135/2023f/hw4.html

This is the *starter code* notebook for Problems 1 and 2, looking at sentiment classification using bag-of-words features of product reviews.

In [39]:
import numpy as np
import pandas as pd
import os
import sys
import time
from random import shuffle

In [40]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [41]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree

In [42]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

In [43]:
# TODO fix to path on your local system
DATA_DIR = os.path.join("data_product_reviews/")

### Load training

In [44]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))

Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [45]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))

Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [46]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))

Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [47]:
vocab_list = x_tr_df.columns.tolist()

In [48]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)

good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [49]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])

In [50]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])

In [51]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [52]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', 
    max_depth=3, min_samples_split=2, min_samples_leaf=1, 
    random_state=101)

# simple_tree_random = sklearn.tree.DecisionTreeClassifier(
#     criterion='gini', 
#     max_depth=3, min_samples_split=2, min_samples_leaf=1, 
#     random_state=101, splitter = 'random')

# simple_tree_best = sklearn.tree.DecisionTreeClassifier(
#     criterion='gini', 
#     max_depth=3, min_samples_split=2, min_samples_leaf=1, 
#     random_state=101, splitter = 'best')

### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [53]:
simple_tree.fit(x_tr_NF, y_tr_N)

# simple_tree_random.fit(x_tr_NF, y_tr_N)

# simple_tree_best.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=3, random_state=101)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [54]:
# Call pretty_print_sklearn_tree on simple_tree, providing feature_names=vocab_list
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)

# pretty_print_sklearn_tree(simple_tree_random, feature_names=vocab_list)

# pretty_print_sklearn_tree(simple_tree_best, feature_names=vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['movie'] <= 0.50?
    

## 1B : Find best Decision Tree with grid search

In [55]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)

In [56]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )

**TODO Build the Grid Search** in the next coding cell

Follow the 1B instructions carefully

In [57]:
# Use grid search for tree classifier
tree_grid_searcher = sklearn.model_selection.GridSearchCV(estimator=base_tree, param_grid=tree_hyperparameter_grid_by_name,scoring='balanced_accuracy',cv=my_splitter, return_train_score=True, refit=False)

### Do the search!


In [58]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [59]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    # Will be here if you called fit on tree_grid_searcher succesfully
    tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        tree_search_results_df.shape[0], elapsed_time_sec))

Grid search of  12 configurations done after   81.0 sec


### Display search results

This block will make a pretty printed table of the results of your grid search

In [60]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    tree_keys = ['param_max_depth', 'param_min_samples_leaf']
    tree_search_results_df.sort_values(tree_keys, inplace=True)
    display(tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']])

,param_max_depth,param_min_samples_leaf,mean_train_score,mean_test_score,rank_test_score,mean_fit_time
0,2,1,0.6404,0.6339,10,1.5089
1,2,3,0.6404,0.6339,10,1.4426
2,2,9,0.6404,0.6339,10,1.2823
3,8,1,0.7279,0.6989,7,3.7573
4,8,3,0.7232,0.6978,8,3.6614
5,8,9,0.7123,0.6962,9,3.7118
6,32,1,0.9187,0.7325,3,9.4431
7,32,3,0.8752,0.7349,2,9.0529
8,32,9,0.8121,0.7197,6,8.4080
9,128,1,0.9981,0.7366,1,14.7607


In [61]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")

Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 1, 'random_state': 101}


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [62]:
# best_tree = base_tree 

best_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', max_depth = 128, min_samples_split=2, min_samples_leaf=1)
# Call set_params using the best_params_ found by your searcher
best_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=128)

### Interpret the best decision tree

In [25]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

The binary tree structure has 1537 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    5 are leaves
- depth   5 has   22 nodes, of which    6 are leaves
- depth   6 has   32 nodes, of which   13 are leaves
- depth   7 has   38 nodes, of which   21 are leaves
- depth   8 has   34 nodes, of which   16 are leaves
- depth   9 has   36 nodes, of which   14 are leaves
- depth  10 has   44 nodes, of which   21 are leaves
- depth  11 has   46 nodes, of which   26 are leaves
- depth  12 has   40 nodes, of which   17 are leaves
- depth  13 has   46 nodes, of which   25 are leaves
- depth  14 has   42 nodes, of which   25 are leaves
- depth  15 has   34 nodes, of which   17 are leaves
- depth  16 has   34 nodes, of which   16 are leaves
- depth  17 has   36 nodes, of which   18 are leaves
- de

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [26]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

### Fit the forest

**TODO Train on the training set** in the next coding cell

In [27]:
# Call fit, like: `simple_forest.fit(None, None)`
simple_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(max_depth=3, random_state=101)

## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [28]:
# Sort features by importance
importRating, featureName = list(zip(*sorted(zip(simple_forest.feature_importances_, vocab_list),reverse=True)))

# Display top 10 most important
simpleForestImportFeats = pd.DataFrame({'feature': featureName, 'importance': importRating})
display(simpleForestImportFeats[0:10])

# print(simpleForestImportFeats[0:10].style.to_latex())

lowImport = np.less(importRating,0.00001)

print('%i features available with importance < 0.00001' % lowImport.sum())

unimpFeats = np.array(featureName)[lowImport]
unimportRating = np.array(importRating)[lowImport]

# Display top 10 most important and 10 random unimportant features
simpleForestImportFeatsTop = pd.DataFrame({'Important Feature': featureName[0:10], 'Importance': importRating[0:10], 'Unimportant Feature': unimpFeats[47:7248:790], 'Importance ': unimportRating[47:7248:790]})
display(simpleForestImportFeatsTop)
# print(simpleForestImportFeatsTop.style.to_latex())

,feature,importance
0,return,0.0335
1,excel,0.0295
2,great,0.0293
3,worst,0.0284
4,poor,0.0269
5,disappoint,0.0244
6,i_love,0.0185
7,your_money,0.0180
8,don't,0.0176
9,bore,0.0164


7288 features available with importance < 0.00001


,Important Feature,Importance,Unimportant Feature,Importance
0,return,0.0335,you_in,0.0
1,excel,0.0295,to_understand,0.0
2,great,0.0293,the_answer,0.0
3,worst,0.0284,return_to,0.0
4,poor,0.0269,nothing_else,0.0
5,disappoint,0.0244,japan,0.0
6,i_love,0.0185,happened_to,0.0
7,your_money,0.0180,don't_see,0.0
8,don't,0.0176,bias,0.0
9,bore,0.0164,a_major,0.0


In [2]:
# REMOVE ZERO and Sort features by importance
importRating, featureName = list(zip(*sorted(zip(simple_forest.feature_importances_, vocab_list),reverse=True)))

# Display top 10 most important
simpleForestImportFeats = pd.DataFrame({'feature': featureName, 'importance': importRating})
display(simpleForestImportFeats[0:20])

# print(simpleForestImportFeats[0:10].style.to_latex())

lowImport = np.less(importRating,0.00001)

print('%i features available with importance < 0.00001' % lowImport.sum())

unimpFeats = np.array(featureName)[lowImport]
unimportRating = np.array(importRating)[lowImport]

# Display top 10 most important and 10 random unimportant features
simpleForestImportFeatsTop = pd.DataFrame({'Important Feature': featureName[0:10], 'Importance': importRating[0:10], 'Unimportant Feature': unimpFeats[47:7248:790], 'Importance ': unimportRating[47:7248:790]})
display(simpleForestImportFeatsTop)
# print(simpleForestImportFeatsTop.style.to_latex())

NameError: name 'simple_forest' is not defined

## 2C: Best Random Forest via grid search

Follow the instructions and using what you learn in 1B to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.)

If yours runs significantly longer, try this out on Google Colab instead.

In [65]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)


In [66]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[100],
    random_state=[101],
    )

In [31]:
# construct a GridSearchCV object like you did above.

forest_searcher = sklearn.model_selection.GridSearchCV(estimator=base_forest, param_grid=forest_hyperparameter_grid_by_name,scoring='balanced_accuracy',cv=my_splitter, return_train_score=True, refit=False)


### Do the search!

In [32]:
start_time_sec = time.time()
forest_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

In [33]:
if hasattr(forest_searcher, 'cv_results_'):
    # Will be here if you called fit on forest_searcher succesfully
    forest_search_results_df = pd.DataFrame(forest_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        forest_search_results_df.shape[0], elapsed_time_sec))


Grid search of  10 configurations done after  104.8 sec


### Display search results

In [34]:
if hasattr(forest_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    forest_keys = ['param_max_depth', 'param_max_features']
    forest_search_results_df.sort_values(forest_keys, inplace=True)
    display(forest_search_results_df[forest_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']])


,param_max_depth,param_max_features,mean_train_score,mean_test_score,rank_test_score,mean_fit_time
0,16,3,0.9139,0.7490,10,0.5764
1,16,10,0.9294,0.8333,6,1.0477
2,16,33,0.9244,0.8462,2,2.5068
3,16,100,0.9170,0.8342,5,6.3941
4,16,333,0.8725,0.7854,9,19.1518
5,32,3,0.9762,0.7926,8,0.9039
6,32,10,0.9721,0.8471,1,1.8854
7,32,33,0.9690,0.8373,4,4.5884
8,32,100,0.9677,0.8392,3,15.8568
9,32,333,0.9540,0.8198,7,48.6250


### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [64]:
print(forest_searcher.best_params_)

best_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100, criterion='gini', max_depth=32, max_features=10, min_samples_leaf=1,random_state=101)

best_forest.fit(x_tr_NF, y_tr_N)

{'max_depth': 32, 'max_features': 10, 'min_samples_leaf': 1, 'n_estimators': 100, 'random_state': 101}


RandomForestClassifier(max_depth=32, max_features=10, random_state=101)

In [36]:
# for tree in best_forest:
#     print(tree.max_depth)

print(len(vocab_list))

7729


In [37]:
# Sort features by importance
importRating, featureName = zip(*sorted(zip(best_forest.feature_importances_, vocab_list),reverse=True))

# Display top 20 most important
bestImportFeats = pd.DataFrame({'feature': featureName, 'importance': importRating})
display(bestImportFeats[0:20])
print(bestImportFeats[0:20].style.to_latex())

,feature,importance
0,great,0.0075
1,bad,0.0059
2,disappoint,0.0057
3,excel,0.0057
4,waste_of,0.0055
5,your_money,0.0049
6,poor,0.0047
7,bore,0.0041
8,return,0.0041
9,worst,0.0039


\begin{tabular}{llr}
 & feature & importance \\
0 & great & 0.007526 \\
1 & bad & 0.005939 \\
2 & disappoint & 0.005716 \\
3 & excel & 0.005700 \\
4 & waste_of & 0.005503 \\
5 & your_money & 0.004885 \\
6 & poor & 0.004723 \\
7 & bore & 0.004114 \\
8 & return & 0.004061 \\
9 & worst & 0.003867 \\
10 & waste & 0.003833 \\
11 & easy & 0.003564 \\
12 & perfect & 0.003303 \\
13 & easy_to & 0.003267 \\
14 & wonder & 0.003171 \\
15 & horribl & 0.002967 \\
16 & the_worst & 0.002693 \\
17 & the_best & 0.002686 \\
18 & an_excellent & 0.002639 \\
19 & love & 0.002462 \\
\end{tabular}



### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|

In [63]:
simpleTree_tr = sklearn.metrics.balanced_accuracy_score(y_tr_N, simple_tree.predict(x_tr_NF))
simpleTree_va = sklearn.metrics.balanced_accuracy_score(y_va_T, simple_tree.predict(x_va_TF))
simpleTree_te = sklearn.metrics.balanced_accuracy_score(y_te_T, simple_tree.predict(x_te_TF))


bestTree_tr = sklearn.metrics.balanced_accuracy_score(y_tr_N, best_tree.predict(x_tr_NF))
bestTree_va = sklearn.metrics.balanced_accuracy_score(y_va_T, best_tree.predict(x_va_TF))
bestTree_te = sklearn.metrics.balanced_accuracy_score(y_te_T, best_tree.predict(x_te_TF))


simpleForest_tr = sklearn.metrics.balanced_accuracy_score(y_tr_N, simple_forest.predict(x_tr_NF))
simpleForest_va = sklearn.metrics.balanced_accuracy_score(y_va_T, simple_forest.predict(x_va_TF))
simpleForest_te = sklearn.metrics.balanced_accuracy_score(y_te_T, simple_forest.predict(x_te_TF))


bestForest_tr = sklearn.metrics.balanced_accuracy_score(y_tr_N, best_forest.predict(x_tr_NF))
bestForest_va = sklearn.metrics.balanced_accuracy_score(y_va_T, best_forest.predict(x_va_TF))
bestForest_te = sklearn.metrics.balanced_accuracy_score(y_te_T, best_forest.predict(x_te_TF))

comparisonDF = pd.DataFrame({'Method': ['simple Tree','best Tree', 'simple RandomForest','best RandomForest'],'Max Depth': [3,128,3,32], 'num trees': [1,1,100,100],'Training': [simpleTree_tr,bestTree_tr,simpleForest_tr,bestForest_tr],'Validation': [simpleTree_va,bestTree_va,simpleForest_va,bestForest_va],'Test': [simpleTree_te,bestTree_te,simpleForest_te,bestForest_te]})

display(comparisonDF)
print(comparisonDF.style.to_latex())


,Method,Max Depth,num trees,Training,Validation,Test
0,simple Tree,3,1,0.6458,0.6446,0.6458
1,best Tree,128,1,0.9965,0.7380,0.7413
2,simple RandomForest,3,100,0.8164,0.7971,0.7731
3,best RandomForest,32,100,0.9721,0.8471,0.8157


\begin{tabular}{llrrrrr}
 & Method & Max Depth & num trees & Training & Validation & Test \\
0 & simple Tree & 3 & 1 & 0.645773 & 0.644560 & 0.645849 \\
1 & best Tree & 128 & 1 & 0.996531 & 0.738045 & 0.741307 \\
2 & simple RandomForest & 3 & 100 & 0.816448 & 0.797119 & 0.773097 \\
3 & best RandomForest & 32 & 100 & 0.972117 & 0.847134 & 0.815661 \\
\end{tabular}

